In [ ]:
%run -i ../python/common.py
UC_SKIPTERMS=True
%run -i ../python/ln_preamble.py

# SLS Lecture 14 : Assembly using the OS

- Review of Processes
- Review of Systems Calls
- I/O - Assembly "Hello World" and read
- Process Address Spaces 
  - Dynamic Memory : Adding and Growing our heap

In [ ]:
# setup for sumit examples
appdir=os.getenv('HOME')
appdir=appdir + "/syscalls"
#print(movdir)
output=runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";mkdir " + appdir + 
             ";cp ../src/Makefile ../src/setup.gdb  ../src/hello.s ../src/read.s ../src/brk.s ../src/exploringASlayout{1..5}.s " + appdir)

display(Markdown('''
- create a directory `mkdir syscalls; cd syscalls`
- copy examples
- add a `Makefile` to automate assembling and linking
    - we are going run the commands by hand this time to highlight the details
- add our `setup.gdb` to make working in gdb easier
- normally you would want to track everything in git
'''))
TermShellCmd("ls " + appdir)

## Review of Processes

### Processes and the Kernel
<center>
<img src="../images/UnixRunning.png">
    </center>

<center>
    <img src="../images/Processes/Processes.004.png">
</center>

<center>
    <img src="../images/Processes/Processes.005.png">
</center>

## Review of System Calls - From Lecture 08

In [ ]:
display(Markdown(htmlFig("../images/kerneldef.png")))

Remember this...

**The "Kernel" -- Unique to Every OS**

1. Bootstraps the HW and has direct access to all of it
1. Bottom layer that enables other programs to run
2. <font style="color:red;"> **A unique collection of functions that programs can invoke** </font>

Not useful on its own only useful and accessed by running other programs.

<center>
<img src="../images/Processes/Processes.051.png">
</center>

### Intel `syscall`

On Intel the instruction is `syscall`

<img src="../images/syscallmp.png">



### The OS System Calls

Each OS Kernel provides a set of calls that a process can invoke using the `syscall` instruction on an Intel based computer

The Linux Kernel supports a very large number of system calls each is identified by a unique number that must be placed in `RAX` prior to executing the `syscall` instruction.  Additional arguments are passed in by setting other registers.  

With each version of the Kernel the table of calls changes.  Here is one site that provides a list

### LINUX SYSTEM CALLS

- reading some man pages `man syscalls` and `man syscall` we find that
  - to exit we must place `60` in `rax`
  - and that the value we want to set as our exit status code in `rdi`
- each system call will accept arguments in various registers 
- NOTE: **On INTEL 64bit processors `rcx` will be overwritten by the syscall instruction**
- On INTEL 64bit processors Linux system calls will return a value back in `rax` and possibly `rdx`
  - `rax` and possibly `rdx` will be overwritten by a system call

In [ ]:
TermShellCmd("man syscalls | head -20", markdown=False)
TermShellCmd("man syscall | head -20", markdown=False)

In [ ]:
display(IFrame("https://filippo.io/linux-syscall-table/", height=600, width="100%"))

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/exit_bb_bb.s", 
    lang="gas", 
    title="",
    h="100%", 
    w="100%"
)))

## I/O

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.017.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.018.png">
</center>

### Review from Lecture 3

<center>
    <img src="../images/Processes/Processes.006.png">
</center>

<center>
    <img src="../images/Processes/Processes.007.png">
</center>

<center>
    <img src="../images/Processes/Processes.008.png">
</center>

<center>
    <img src="../images/Processes/Processes.009.png">
</center>

<center>
    <img src="../images/Processes/Processes.010.png">
</center>

<center>
    <img src="../images/Processes/Processes.011.png">
</center>

<center>
    <img src="../images/Processes/Processes.012.png">
</center>

<center>
    <img src="../images/Processes/Processes.013.png">
</center>

<center>
    <img src="../images/Processes/Processes.015.png">
</center>

### Finally "Hello World in Assembly"

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/hello.s", 
    lang="gas", 
    title="<b>CODE: asm - hello.s",
    h="100%", 
    w="107em"
)))

In [ ]:
TermShellCmd("make hello", cwd=appdir, prompt='')

#### At long last we have a program that we don't need the debugger for!

In [ ]:
TermShellCmd("./hello; echo $?", cwd=appdir)

### How about input -- remember standard input

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/read.s", 
    lang="gas", 
    title="<b>CODE: asm - read.s",
    h="100%", 
    w="107em"
)))

### Blocking: System calls can "block" your program 

By default the read system call will not return until some data is read or an error occurs.
So when we run the read example it will wait for the read to return before it goes on to exit.
This waiting on a system call is called "blocking"


### There are many more calls for doing I/O

But we can now 
1. transfer bytes from our address space to a "file" - write system call
2. transfer bytes from a "file" to our address space  - read system call
3. connect (and possibly create) files to our address space for reading and writing - open system call
4. disconnect a file from our address space - close system call


In [ ]:
display(IFrame("https://filippo.io/linux-syscall-table/", height=600, width="100%"))

### Exercises

Change the `usesum` program to:
- exit properly 
- allocate memory for the data
- read the data in from standard in
- write the binary result to standard out
  - use a shell command like `od -l -A10` to convert the output to ascii
- open the binary file of numbers and read them in
- can you figure out how to get the command line arguments?
- Write a new program that uses the random instruction to create a data file


## Process Address Spaces

### Address Space organizes Memory of a Process

<center>
    <img src="../images/ProcessContexts.png">
</center>

**Remember**

1. The binaries we create using the assembler and linker are Executables

2. When we "run" our executables via the shell it creates a new process context and our binary is loaded as the initial memory "image".

3. The memory and register values of the process are unique to each process and change as the instructions of our binary "execute"

### Assembly Programming as Process Address Space Image creation

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.003.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.004.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.005.png">
</center>

### The Details

### Process Address Space 

The Memory of a Process is organized as an (Virtual) Address Space 

1. Each possible location of a byte is identified by a unique address (number) 
2. To associate a Region (range of continues addresses) with actual memory requires a "mapping"
    - mappings associate a Region with Memory and a possible source of values
    - mappings can restrict what kind of access can be made to region 
        1. fetch for execute: x
        2. read: r
        3. write: w

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-001.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-002.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-003.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-004.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-005.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-006.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-007.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-008.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-009.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-010.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-011.png">
</center>

<hr>

## Summary of how we "loading" a process address from a binary 

<hr>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.012.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.016.png">
</center>

## Code and instructions for exploring the Process Address Space layout

In the lecture notes at this point you will find code and instructions for exploring the address space layout

1. How we control the mappings based on the sections we add
2. How we can add and shrink heap memory using the `brk` system call

To do the exploration we use :
1. The `nm` command that print's the symbol table of a binary.  As such we can use it to see what addresses the linker placed our bytes at.
2. A special file that the OS provides that describes the address space of a running process call the `maps` file. 

Detail are provide in the notes.  Don't forget you can also use gdb to probe the various mappings you discover.  
Eg. You can use the `x` command to examine the bytes at the addresses of the memory areas you find in the maps file.



### Exploring the Address Space of a process with Code

- Let's explore the address space of a running program 
- We will use 5 versions of a program the progressively adds more regions to the address space
-  The programs will wait for input before exiting so that we can use the OS provided file for check what the process address space looks like



#### Version 1: text and data

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/exploringASlayout1.s", 
    lang="gas", 
    title="<b>CODE: asm - exploringASlayout1.s",
    h="100%", 
    w="107em"
)))

In [ ]:
TermShellCmd("make exploringASlayout1", cwd=appdir, prompt='')

Use the nm command to see where the linker placed things:

In [ ]:
TermShellCmd("nm exploringASlayout1", cwd=appdir, prompt='')

- Now in one terminal run the binary -- start a process from it

```
./exploringASlayout1
```

- It will hang on the read so that we have time to explore what the process's address space looks like.

- In another terminal use ps and grep to find the process id of the new process

```
ps auxgww | grep exploring
```

Now we can use the process id to find the special file that the OS provides for us to examine the address space of a process.

```
 cat /proc/<pid>/maps
```

You can also use gdb to to explore the regions of memory that the maps file tells us are present.  You can also compare what is in memory to what you find in the file.

When you are done you can press return in the terminal that the program in running.  This will send a newline to the standard input of the program.  This will cause the read to return and the program will go on to exit.


You can find lots of information ab out the OS provided `proc` directory in the manual `man proc`

In [ ]:
TermShellCmd("man proc | head -30", markdown=False)

In the manual page there is a section on the `maps` file specifically that tells us the details of what information it presents regarding the mappings for a running process.

In [ ]:
TermShellCmd("man proc | grep -A40 /maps | head -41", markdown=False)

#### Version 2:  adding rodata

Read-only data section `.rodata`

```nasm
  ##### DATA SECTION : read and writable memory s
    .section .data
    .global rwdata
rwdata: .fill 4096,1,0xff           # 4096 bytes each initialized with a value of 0xff
```

What would happen if we put all our data in the `.rodata` section?... try it and find out


Here is the code

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/exploringASlayout2.s", 
    lang="gas", 
    title="<b>CODE: asm - exploringASlayout2.s",
    h="100%", 
    w="107em"
)))

Repeat the steps from version 1 to see what has changed.

#### Version 3:  adding bss

The bss section is used to create a mapping for all the data that we need memory for but don't need to specify an initial value that is not zero.

Read-Write memory  `.bss`

```nasm
    ##### BSS DATA SECTION: read and write automaitically added and filled with zero values
    .section .bss
    .global zerodata 
zerodata:   .fill 4096,1
    # The above could be replaced with the single line: .comm zerodata, 4096, 1
```

Note the `.comm` directive is a short cut that switch the section to .bss adds the symbol and makes it global

Here is the code

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/exploringASlayout3.s", 
    lang="gas", 
    title="<b>CODE: asm - exploringASlayout3.s",
    h="100%", 
    w="107em"
)))

Repeat the steps from version 1 and see what what has changed compared to versions 1 and 2

#### Version 4: And and Grow our Heap

Now let's explore how while our program is running we can get more memory added to our process.


The `brk` system call - move "break pointer"

- initially the data segment of your process will end at some location based on what data sections you defined
  - this location is called the program break pointer 
  - the location where valid data memory ends and the un-allocated virtual address space
- with `brk` you can set the end to a value bigger than it starting location
- you can also shrink the break pointer back to remove memory

This area is called the HEAP

1. call `brk` with argument of 0 and Linux kernel returns current break address
2. call `brk` "with a reasonable address" and the kernel will allocated or de-allocated memory for us

- in reality once we move to C we never directly call "brk" rather we will use a library routines called : `malloc` and `free` which will call `brk` for us.

As usual you can find more info in the manual 

In [ ]:
TermShellCmd("man brk | head -37", markdown=False)

You can find the system call number and what the arguments are here:

In [ ]:
display(IFrame("https://filippo.io/linux-syscall-table/", height=600, width="100%"))

**Lets add this code**

What does it do?

```gas
    # Now add a Heap
    # get current break pointer
    xor rdi, rdi               # pass 0 to brk (invalid request)
    mov rax, 12                # brk syscall number 12
    syscall                    # call brk with 0

    mov r15, rax               # keep a copy of where our new memory starts
                               # so we can use it later

    # now add some memory by requesting to increase the break
    # address by 32 bytes
    mov     rdi, rax            # mov current break into rdi
    add     rdi, 4096           # ask for 4096 bytes rdi=rdi+4096
    mov     rax, 12             # brk sycall 12
    syscall
```

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/exploringASlayout4.s", 
    lang="gas", 
    title="<b>CODE: asm - exploringASlayout4.s",
    h="100%", 
    w="107em"
)))

We have added more code so that you can pause the program before and after it makes the calls to brk so that you can see that heap get added.

1. start the program running
2. follow the same steps as the other versions to look at the address space layout
3. then press enter -- this will let the program continue.  It will then run the code we added to add the heap area and pause again
4. again examine the address space layout -- do you see the difference?
5. press enter again -- this will again let the program continue and it will exit


#### Version 5: giving memory back -- shrink the heap back down

In this final version we add a final step to the program to shrink the heap back to its original size (0).  This removes it from our process and give the memory back to the OS.

Here is the code we add -- remember when we first called `brk` we stored the original location in `r15`

```
    # remove memory --  shrink heap back down to its original
    mov rdi, r15               # set break pointer back to the original location
    mov rax, 12                # brk syscall number 12
    syscall
```    


In [ ]:
display(Markdown(FileCodeBox(
    file="../src/exploringASlayout5.s", 
    lang="gas", 
    title="<b>CODE: asm - exploringASlayout5.s",
    h="100%", 
    w="107em"
)))

Again we added another read so that you can explore what the address space looks like after we shrink the heap but before we exit.  

So that's it -- we have seen how memory gets added to the process based on what sections we added in our source code and how we can make calls to the OS to  add and remove "heap" memory while our program is running.
There is a more advanced call in UNIX called `mmap` but we will leave it's discussion for another time.
